In [53]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import torch

In [54]:
torch.manual_seed(42)

In [55]:
torch.manual_seed(42)
n = 1000
t = np.linspace(0, 1, n)
elevation_true = 0.8 * ((np.sin(50*t) + np.sin(10*t))/2)+ 0.1 * t   # Profilo di elevazione reale
elevation_measured = elevation_true + 0.1 * torch.rand(n).numpy()  # Dati misurati con rumore

In [56]:
# Operatori differenziali discreti
D = np.eye(n) - np.eye(n, k=-1)
D = D[:-1, :]  # Prima derivata discreta

D2 = np.eye(n) - 2 * np.eye(n, k=1) + np.eye(n, k=2)
D2 = D2[:-2, :]  # Seconda derivata discreta

In [57]:
ds = []

In [32]:
for i in range(1000):

	# Simulazione dei dati


	# Variabile di ottimizzazione
	x = cp.Variable(n)



	# Parametri del problema
	rho_param = np.random.rand()  # Peso della regolarizzazione sulla pendenza
	kappa_max = np.random.rand()   # Limite sulla curvatura

	# Funzione obiettivo
	objective = cp.Minimize(cp.sum_squares(x - elevation_measured))

	# Vincoli
	constraints = []

	# Vincoli sulla curvatura
	constraints += [cp.sum_squares(D2 @ x) <= kappa_max*np.linalg.norm(D2@elevation_measured)**2]
	constraints += [cp.sum_squares(D @ x) <= rho_param*np.linalg.norm(D@elevation_measured)**2]

	# Vincoli sui limiti di elevazione (se noti)
	elevation_min = np.min(elevation_measured) - 0.1
	elevation_max = np.max(elevation_measured) + 0.1
	#constraints += [x >= elevation_min]
	#constraints += [x <= elevation_max]

	# Risoluzione del problema
	prob = cp.Problem(objective, constraints)
	prob.solve()

	# Soluzione ottimale
	x_hat = x.value

	if prob.status == 'optimal': ds.append([np.array(rho_param), np.array(kappa_max), x.value, constraints[0].dual_value, constraints[1].dual_value])

In [33]:
(x.value).shape

(100,)

In [34]:
constraints[0].dual_value

array([5.8307838])

In [35]:
np.concatenate([x.value, constraints[0].dual_value]).shape

(101,)

In [36]:
constraints[1]

Inequality(Expression(CONVEX, NONNEGATIVE, ()))

In [37]:
constraints[1].dual_value

array([2.5360862e-10])

In [38]:
constraints[0].dual_value

array([5.8307838])

In [39]:
np.linalg.norm(D2@x.value)**2 - kappa_max*np.linalg.norm(D2@elevation_measured)**2

-7.760286857561027e-11

In [40]:
np.linalg.norm(D@x.value)**2 - rho_param*np.linalg.norm(D@elevation_measured)**2

-0.6953843297786368

In [41]:
D.shape

(99, 100)

In [42]:
np.linalg.norm(2*(x.value - elevation_measured) + 2*constraints[1].dual_value*(D.T@(D@x.value))+ 2*constraints[0].dual_value*(D2.T@(D2@x.value)))

0.0001297525370904719

In [43]:
constraints[1].dual_value*(np.linalg.norm(D@x.value)**2 - rho_param*np.linalg.norm(D@elevation_measured)**2)

array([-1.7635546e-10])

In [44]:
constraints[0].dual_value*(np.linalg.norm(D2@x.value)**2 - kappa_max*np.linalg.norm(D2@elevation_measured)**2)

array([-4.52485549e-10])

In [45]:
ds

[[array(0.14962791),
  array(0.40143904),
  array([ 0.09670982,  0.19457408,  0.27451816,  0.32933923,  0.34495637,
          0.33125475,  0.29598533,  0.25962785,  0.23235357,  0.22602094,
          0.25809121,  0.3164163 ,  0.39561133,  0.47620681,  0.54429785,
          0.58214077,  0.58464479,  0.5450857 ,  0.47524203,  0.39336756,
          0.31198219,  0.25165259,  0.22060866,  0.22327205,  0.24636423,
          0.28588974,  0.32249574,  0.33858386,  0.3246916 ,  0.2725173 ,
          0.19439024,  0.09087218, -0.00892374, -0.09761423, -0.15716608,
         -0.17758202, -0.16695256, -0.13232699, -0.0937389 , -0.06401055,
         -0.05482843, -0.07774894, -0.12822912, -0.20129006, -0.28448075,
         -0.35822237, -0.40053365, -0.40862264, -0.37884057, -0.32125893,
         -0.24344106, -0.16085092, -0.09389308, -0.05096444, -0.04530535,
         -0.06770333, -0.1062097 , -0.14344185, -0.16068188, -0.15115024,
         -0.10062653, -0.01846687,  0.08829871,  0.19635843,  0.292346

In [46]:
from pickle import dump
dump(ds, open("smooth.pkl", "wb"))

In [47]:
x.value.shape

(100,)

In [48]:
D2.shape

(98, 100)

In [49]:
np.linalg.norm(D2@elevation_measured)**2

1.084829414601063

In [50]:
np.linalg.norm(D2 @ elevation_true)**2

0.49741764541027556

In [51]:
np.max(D2 @ elevation_true)

0.10242222754502994

In [52]:
np.linalg.norm(D2 @ x.value)

0.5000576257196349